In [2]:
import sys
import os

# إضافة مجلد الجذر (IR-project) لمسارات البحث
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print("Added to sys.path:", project_root)

from query_processor import query_service

dataset = "antique"   # أو "beir"
query = "Tell me about the Roman Empire."

vector = query_service(dataset, query)
print("Vector shape:", vector.shape)
print("Vector sample values:", vector.toarray()[0][:10])


Added to sys.path: c:\Users\HP\IR-project


AttributeError: Can't get attribute 'tokenizer' on <module 'TF_IDF'>

In [4]:
import sys
import os

# أضف مجلد المشروع (IR-project) إلى مسارات البحث
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)

print(os.listdir(project_root))


['.git', '.gitignore', '.idea', 'dataBases', 'embeddings.joblib', 'fix.py', 'indexing', 'IR.ipynb', 'nltk_data', 'query_processing', 'storage', 'store.ipynb', 'text_processing', 'vectorize', '__pycache__']
